<a href="https://colab.research.google.com/github/gucher/Remote_Grave_Digger/blob/master/Remote_Grave_Digger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How I built a remote controlled Grave Digger with a Raspberry Pi

*This is part 1 of a series in which I take a standard RC car and add numerous technology and software upgrades in pursuit of a fully autonomous Grave Digger.*

*I am a BEGINNER in DIY electronic and robotics, this post is not intended as a tutorial or to be used for advice.*

*In Part 1 we will explore:*

*   Deconstruction
*   Wiring and framework
*   Battery source
*   Powering external components
*   Python & curses library
*   Motor and speed control (PWM)

## Original Car

In [8]:
%%html

<img src=https://drive.google.com/uc?id=13omzro0UhENkenbvYvJRBl4YPV9l8G-u width="500">

#### At first I was looking to find a smaller car with a wide range of motion, but I laid eyes on this beauty at Big Lots and couldn't resist the sale price of $20.

For those who don't know, here is the real Grave Digger.

In [6]:
%%html
<img src=https://drive.google.com/uc?id=1f-T7IcAiLXY8NO_C5GRtvi9bUZGeRYVq width="500">

## Deconstruction

Removing the body exposes a plastic frame and two sets of wiring to the front and rear axles.

In [10]:
%%html
<img src=https://drive.google.com/uc?id=1Jt_gmCn6PJC8Fm7YF5gisILsQaJpuvX4 width="500">

The bottom of the car holds a battery pack with power on/off switch. Front and rear axles are removable.

In [11]:
%%html
<img src=https://drive.google.com/uc?id=1h5xrLZJFlOmDnFv1xUKZjXejvJf3xGiU width="500">

Default controller hooked up to battery pack and DC motors.

In [12]:
%%html
<img src=https://drive.google.com/uc?id=12mlGh7Xx7VV-Azu4HwA2Kw8vAb2Jw79m width="500">

Starting to remove extra parts and components.

*I explored the idea of switching to new tires but decided against it as the project went on.*

In [13]:
%%html
<img src=https://drive.google.com/uc?id=19-H_gTRc5r_uFdRB3mRfajZYaxSFyaWi width="500">

Here's the RC car broken down. I was surprised with how simple these cheap RC cars are... They consist of very basic part and components.

In [14]:
%%html
<img src=https://drive.google.com/uc?id=1ZXl4hqj67dI5kv8Kw42NUoPqW9bTbX2c width="500">

To determine which GPIO pins control F/B and L/R control, I used an extra DC motor for testing.

In [15]:
%%html
<img src=https://drive.google.com/uc?id=1Q-495oJAm4tE2RNQASTLMjkM5kY-43Y3 width="500">

*I am not an electonics expert (this is my first real project), so I relied on tutorials and infographics a lot.*

Setup for controlling a DC motor with L298N.

In [16]:
%%html
<img src=https://drive.google.com/uc?id=1ViGeNgD4M08J_eAbHheEs4iY6LE7O7e2 width="500">

To lower the speed at which the rear motor cycles, I used PWM (Pulse Width Modulation) capable GPIO pins.

In [17]:
%%html
<img src=https://drive.google.com/uc?id=1aooWqVkVjGv4NIorVhxvBA799YnOCFoi width="500">

Once the L298N was wired to front and rear motors, I affixed a rectangle of cardboard to the bottom as a placeholder to rest all the components on.

In [18]:
%%html
<img src=https://drive.google.com/uc?id=1dle7K6xfJX2IS-xWxRvbaYDILRfbRPR8 width="500">

Here is the car with motor setup and battery power.

In [19]:
%%html
<img src=https://drive.google.com/uc?id=1Jz27PFRd7CLC524b6OPJrqiqvlieS1b_ width="500">

*I connected a simple On/Off switch and placed the battery pack at the base. Later in the project I switched out the battery pack pictured above and drew power straight from the L298N, which was powered by a separate 8AA pack.*

In [20]:
%%html
<img src=https://drive.google.com/uc?id=1LQsxerijTHog5nP7tkvWK1vBc0DmJVCp width="500">

For testing, I neglected any cable (wire) management and opted to squeeze everything under the FLIMSY plastic frame. Here's how it turned out without the shell attached (resting in background).

*UPDATE: As noted at the end, this posed a serious issue for airflow and adding additional components (distance sensors, LEDs, camera). The RPi 3 B+ was swtiched out for a RPiZ and 8AA power for external 5V (same component for charing cell phones).*

In [21]:
%%html
<img src=https://drive.google.com/uc?id=1Hfd_o7ZjdhfZmxTPOz9fdoIuhWI5cnQs width="500">

## Now let's look at some code...

To run all code I booted up a Remote Desktop and programmed locally on the RPi. There are numerous tutorials online to setup RPi RDC and PuTTY.

In [0]:
# import libraries

import RPi.GPIO as GPIO
import time

# curses is used to read (echo) which keys are being pressed
# the keys are then assigned to control each motor via the L298N
# once the key is captured and the signal is sent to the motor, the car will
# power that motor
# this can be as simple or complex as desired
# for my first car, I went with the simplest approach
# PWM is not necessary but I knew I eventually wanted to make this autonomous
# and i didn't want the car running into a wall at full speed
import curses

In [0]:
# I am using BOARD setup here
# For BCM setup GPIO pins will need updated in below code

GPIO.setmode(GPIO.BOARD)

In [0]:
# PWM pins 12, 32, 33, 35
# PWM is only needed for forward and reverse
# setup intializes the pins as either input or output
# initial sets the pin to not output at setup, otherwise
# when powering up the car, a pin will continue
# to output until told otherwise

GPIO.setup(33,GPIO.OUT, initial=0) # <-- initial set to 0
GPIO.setup(35,GPIO.OUT, initial=0) # <-- to ensure motors will not run when RPi is turned on
GPIO.setup(13,GPIO.OUT, initial=0) # <-- depending which pins are used
GPIO.setup(15,GPIO.OUT, initial=0) # <-- initializing may not be needed

In [0]:
# setup curses
# to remove text from displaying in terminal
# when controlling car direction and movement,
# use curses.noecho()

screen = curses.initscr()
# curses.noecho() <-- remove comment depending on preference above

curses.cbreak()
curses.keypad(True)

In [0]:
# create variables for front and backwards control
# these pins were determined to control forward and backwards
# when testing with a DC motor
# the pin # should be used for PWM
# but the wiring may be different on other cars/DC motors

f = GPIO.PWM(33,100) # <-- note the PWM capable pins from above
b = GPIO.PWM(35,100) # <-- set 100 for initial frequency

Quick reference on using PWM in RPi.GPIO

https://sourceforge.net/p/raspberry-gpio-python/wiki/PWM/

In [0]:
# start your engines!

f.start(0) # starting at 0% duty cycle
b.start(0)

# while pin setup was initialized at 0
# I wanted to set duty cycle to 0% to start
# for an autonomous vehicle
# the duty cycle may be set to default at x%

In [0]:
# it's a good idea to include a code break and
# print reminder keys (in addition to stopping the script)

print('Press the [q] key to quit.')
print('Press the [b] key to stop the car.')
print('Press [Ctrl + c] to stop the script.')
time.sleep(5) # <-- for us slow readers

### Use try / finally for exception handling. 

*The try statement has another optional clause which is intended to define clean-up actions that must be executed under all circumstances.*

*A finally clause is always executed before leaving the try statement, whether an exception has occurred or not. When an exception has occurred in the try clause and has not been handled by an except clause (or it has occurred in an except or else clause), it is re-raised after the finally clause has been executed. The finally clause is also executed “on the way out” when any other clause of the try statement is left via a break, continue or return statement.*

Reference the documentation: https://docs.python.org/3/tutorial/errors.html

In [0]:
try:
  while True: # <-- loop will run until broken
    char == screen.getch()
    if char == ord('q'):
      break # code will break and move to finally
    elif char == curses.KEY_DOWN:
#     to adjust the value of duty cycle, use ChangeDutyCycle
#     the value can be between 0% (LOW) and 100% (HIGH)
      b.ChangeDutyCycle(30)
      f.ChangeDutyCycle(0)
#     although f is already set to 0%
#     it is good practice to control DC of each variable when changing
    elif char == curses.KEY_UP:
      b.ChangeDutyCycle(0)
      f.ChangeDutyCycle(30)
    elif char == curses.KEY_LEFT:
      GPIO.output(13,GPIO.HIGH)
      GPIO.output(15,GPIO.LOW)
    elif char == curses.KEY_RIGHT:
      GPIO.output(13,GPIO.LOW)
      GPIO.output(15,GPIO.HIGH)
    elif char == ord('b'): # stops the car (like a car brake, not a code break)
      b.ChangeDutyCycle(0)
      f.ChangeDutyCycle(0)

In [0]:
# restore curses and prepare for shutdown
finally:
  curses.nocbreak(); screen.keypad(0); curses.echo()
#   echo may not be turned on if code from earlier is not altered
  curses.endwin()

In [0]:
# clean up GPIO pins for later
  GPIO.cleanup()

### Put all your code together and run it on remote desktop or PuTTY.

## CHEERS!

In [22]:
%%html
<img src=https://drive.google.com/uc?id=14U5ae88hkDFiQQ180zMkSYBTfbZJnuM0 width="500">

### [UPDATE]

A 1xin square is cut-out on the top body and small 5V fan attached inside. The hack-job to the right is a slit for the camera.

Since I didn't want to take up another 5V port from the GPIO breakout, I rigged up a 9V battery cable to a regulator to get 5V output. This is not a recommended solution -- soldering or splitting and reclamping wires is much safer.

In [23]:
%%html
<img src=https://drive.google.com/uc?id=1RnHmGiD4Ku5y-CM6HOKESwBnlTe0IQwW width="500">

My DIY solution works great for small devices that need 5V but please do not follow the makeshift wiring. And DO NOT use expired batteries (I found a 2008 9V that I tested... bad idea).

WARNING!

**** THE REGULATOR WILL GET VERY HOT ****

**** NEVER USE AN EXPIRED BATTERY ****

For a better and safer DIY solution, check out this YouTube video:
https://www.youtube.com/watch?v=8W-zdo1AVns

#### Switch to Raspberry Pi Zero



The RPi 3 was taking up too much space and there wasn't enough airflow for the components -- *without compromising the asthetic of Grave Digger's body* -- so a switch to the RPi Zero was made.

The RPiZ is much sleeker and is capable of running our code and output to hardware without issues.

Pictured below is the updated setup with a camera and ultrasonic distance sensor. In the next update, I'll be reviewing each of these components and much more. Stay tuned!

In [24]:
%%html
<img src=https://drive.google.com/uc?id=1ddTlvnJEdIEpSDPu7P5TrFtReS4aQD6a width="500">

### In the next update, I'll be attaching Ultrasonic Distance sensors and for object detection and avoidance, as well as a camera for video streaming, and maybe a few other fun things!

## Products and useful links.

Starter pack for RPi 3 Model B+

In [25]:
%%html
<img src=https://drive.google.com/uc?id=1XhA91JFZu5y7KHNW8_BNEc6swgoK9fKl width="500">

### Products Used:

Cooling Fan:
https://www.amazon.com/gp/product/B07D3TVVC7/ref=ppx_yo_dt_b_asin_title_o04_s00?ie=UTF8&psc=1

On Off Switch:
https://www.amazon.com/gp/product/B01N2U8PK0/ref=ppx_yo_dt_b_asin_title_o09_s01?ie=UTF8&psc=1

L298N Motor Driver Controller:
https://www.amazon.com/gp/product/B00XAGRQTO/ref=ppx_yo_dt_b_asin_title_o02_s00?ie=UTF8&psc=1

CanaKit RPi 3 B+:
https://www.amazon.com/gp/product/B07BC567TW/ref=ppx_yo_dt_b_asin_title_o05_s02?ie=UTF8&psc=1

8 AA 12V Battery Case:
https://www.amazon.com/gp/product/B00VE7HBMS/ref=ppx_yo_dt_b_asin_title_o09_s00?ie=UTF8&psc=1

*Some pictures are taken directly from Amazon and/or reference links. Do not claim ownership.*

### Useful Links:

Raspberry gPIo (RPi.GPIO API): https://learn.sparkfun.com/tutorials/raspberry-gpio/python-rpigpio-api

Resistor for LED (Resistor Guide): http://www.resistorguide.com/resistor-for-led/

RPi.GPIO Basics: https://raspi.tv/2013/rpi-gpio-basics-5-setting-up-and-using-outputs-with-rpi-gpio

Here is some documentation on the L298N and PWM setup.

https://www.theengineeringprojects.com/2017/07/introduction-to-l298.html

https://learn.sparkfun.com/tutorials/raspberry-gpio/python-rpigpio-api


*2019-08-05: I utilized many resources throughout this build. I have tried to include all for reference. Additional documentation may be added at a later date.*